## 线程通信
- 通信方法:
  - 多个线程共用进程空间,所以进程的全局变量对进程内的线程均可见.因此使用全局变量  
    通信是线程主要通信方法.
- 注意事项:
  - 线程间通信更容易产生资源争夺,往往需要同步互斥机制保证通信安全

## 线程的同步互斥
- 线程的Event事件

- 操作:
  - e = threading.Event()
  - e.wait([timeout]) 如果e为设置状态则不阻塞,未设置则阻塞
  - e.set() 将e变为设置状态
  - e.clear() 将e设置去除


In [9]:
# thread_event
import threading
from time import sleep

s = None

def bar():
    print('呼叫FOO')
    global s
    s = '天王盖地虎'
    
def foo():
    print('等待口令')
    sleep(2)
    if s == '天王盖地虎':
        print('自己人')
    else:
        print('打死他')
    e.set()
    
e = threading.Event()
        
def fun():
    print('hehe...')
    sleep(1)
    e.wait()
    global s
    s = '小鸡炖蘑菇'
        
t1 = threading.Thread(target=bar)
t2 = threading.Thread(target=foo)
t3 = threading.Thread(target=fun)

t1.start()
t2.start()
t3.start()

t1.join()
t2.join()
t3.join()

呼叫FOO
等待口令
hehe...
自己人


## 线程锁
- lock = threading.Lock() 创建锁
- lock.acquire()  上锁
- lock.release()  解锁
- 操作原理: 重复上锁

In [12]:
# thread_lock
import threading



a = b = 0
lock = threading.Lock()

def value():
    lock.acquire()
    while True:
        if a != b:
            print("a=%d,b=%d"%(a,b))
    lock.release()
            
t = threading.Thread(target=value)
t.start()


while True:
    lock.acquire()
    a += 1
    b += 1
    lock.release()
t.join()


## Python线程的GIL问题(全局解释器锁)
- python -->>支持多线 -->> 同步互斥 -->> 加锁 -->> 超级锁(给解释器加锁)
- 无论你启多少个线程，你有多少个cpu, Python在执行的时候会淡定的在同一时刻只允许一个线程运行，  
  擦。。。，那这还叫什么多线程呀？
- Python 的GIL问题 解决方案
  - 尽量使用进程
  - 不使用c作为解释器  其中的JPython就没有GIL 
  - Python线程适用于高延迟的IO操作,网络操作,不适合cpu密集型或者传输速度很快的IO操作
- **注意:线程遇到阻塞会让出解释器**
- 效率测试:
  - 进程的效率明显提高 由于GIL影响线程效率较低

## 进程和线程的区别和联系
1. 两者都是多任务编程的方式,都能够使用计算机多核资源
2. 进程的创建和删除要比线程消耗更多的计算机资源
3. 进程空间独立,数据安全性高,有专门的通信方法
4. 线程使用全局变量通信,更加简单,但是往往需要同步互斥操作
5. 一个进程可以包含我个线程,线程共享进程的资源
6. 进程线程都有自己的特有属性资源,如命令,属性,id等


## 使用场景
- 如果需要创建较多的并发,任务比较简单,线程比较合适
- 如果数据操作和功能比较独立,此时使用进程比较适合
- 使用线程时要考虑到同步互斥的复杂程度
- Python线程需要考虑GIL问题

## 总结
1. 进程线程的特征
2. 进程线程区别和关系
3. 同步互斥的意义,用过什么方法,什么情况下用的
4. 进程间通信方式都知道哪些,有什么特点
5. 僵尸进程怎么处理,线程的GIL问题怎么看
6. 给一个情景,问选择进程还是线程,为什么 

In [14]:
#计算密集

def count(x,y):
    c = 0
    while c < 6000000:
        c += 1
        x += 1
        y += 1

#IO密集
def write():
    f = open('test.txt','w')
    for x in range(1000000):
        f.write("比利时加油\n")
    f.close()

def read():
    f = open('test.txt')
    lines = f.readlines()
    f.close()
'''------------------------------------------------------'''
#单进程的执行效率
from test import *
import time 

# t = time.time()
# for i in range(10):
#     count(1,1)
# print("Line cpu:",time.time() - t)

t = time.time()
for i in range(10):
    write()
    read()
print("Line IO:",time.time() - t)

'''-----------------------------------------------------'''
#多线程CPU
from  test import *
import threading 
import time

jobs = []

t = time.time()
for i in range(10):
    th = threading.Thread\
    (target = count,args = (1,1))
    jobs.append(th)
    th.start()
for i in jobs:
    i.join()
print("Thread cpu:",time.time() - t)

'''----------------------------------------------------'''
#多线程IO
from  test import *
import threading 
import time

def io():
    write()
    read()

jobs = []
t = time.time()
for i in range(10):
    th = threading.Thread(target = io)
    jobs.append(th)
    th.start()
for i in jobs:
    i.join()
print("Thread io:",time.time() - t)

'''-------------------------------------------------'''
#进程cpu
from  test import *
import multiprocessing 
import time

jobs = []

t = time.time()
for i in range(10):
    p = multiprocessing.Process\
    (target = count,args = (1,1))
    jobs.append(p)
    p.start()
for i in jobs:
    i.join()
print("process cpu:",time.time() - t)

'''---------------------------------------------------'''
#多进程io
from  test import *
import multiprocessing
import time

def io():
    write()
    read()

jobs = []
t = time.time()
for i in range(10):
    p = multiprocessing.Process(target = io)
    jobs.append(p)
    p.start()
for i in jobs:
    i.join()
print("Process io:",time.time() - t)

## 服务器模型
- 硬件服务器: 主机  集群
  - 厂商 : IBM  HP   联想    浪潮
- 软件服务器: 编写的服务端程序,依托于硬件服务器运行,提供给用户一定的软件服务
  - 分类:
    - webserver  --> 网站后端程序提供网站请求的后端处理和响应
    - httpserver --> 处理HTTP请求,回复http响应
      - 邮箱服务器 --> 处理邮件
      - 文件服务器 --> 处理文件传输
  - 功能:网络连接,逻辑处理,数据的交互,数据的传输,协议的实现
  - 模型结构:
    - c/s  (客户端服务器模型)
    - b/s  (浏览器服务器模型)
  - 服务器目标:
    - 处理速度快,数据更安全,并发量大
 
- 硬件:更高的配置,集成分布基础,更好的网路速度.更多主机,更好的网路安全性
- 软件:程序占有更少的资源,更稳定的运行效果,更流畅的运行速度,采用更安全更合适的技术

## 基础的服务器模型      
- **循环服务器**:单进程程序,循环接受客户端请求,处理请求,每处理完一个请求再去接受下一个请求.
  - 优点: 实现简单占用资源少
  - 缺点: 无法同时连接多个客户端,当一个客户端长期占有服务器时,形成其他客户端无法操作的情况
  - 使用情况:任务比较短暂,udp套接字更合适

- **并发服务器**: 同时处理多个客户端的任务请求

  - IO并发: IO多路复用  协程
    - 优点: 资源消耗少,效率较高,适用于IO类型服务器
    - 缺点: 不能监控cpu密集型程序,本质是单进程所以不能长期阻塞消息的收发
    
  - 多进程/多线程并发: 为每一个客户端单独提供一个进程或者线程处理请求.由于进程线程执行独立  
    所以对其他进程不会有影响
    - 优点: 客户端可以长期占有服务器,操作不会对其他进程线程产生影响
    - 缺点: 消耗较多资源,不适合有大量客户端的情况
    
## 多进程并发
- 使用fork完成并发
  1. 创建套接字,绑定,监听
  2. 等待接受客户端请求  accept
  3. 创建子进程处理客户端请求  父进程继续等其他客户单连接
  4. 客户端退出则子进程退出

In [ ]:
#fork tcp 并发
from socket import *
import os,sys
import signal

#服务器地址
HOST = '0.0.0.0'
PORT = 8885
ADDR = (HOST,PORT)

#处理客户端请求函数
def client_handle(c):
    print("子进程处理客户端:",c.getpeername())
    while True:
        data = c.recv(1024).decode()
        if not data:
            break
        print(data)
        c.send(b'Receive your message')

#创建tcp监听套接字
s = socket()
s.setsockopt(SOL_SOCKET,SO_REUSEADDR,1)
s.bind(ADDR)
s.listen(5)

#处理僵尸进程
signal.signal(signal.SIGCHLD,signal.SIG_IGN)
print("Listen to 8888.....")
while True:
    try:
        c,addr = s.accept()
    except KeyboardInterrupt:
        s.close()
        sys.exit("服务器退出")
    except Exception as e:
        print(e)
        continue

    #有客户端连接,创建子进程
    pid = os.fork()
    if pid < 0:
        print("创建子进程失败")
    elif pid == 0:
        s.close()  #子进程不需要监听套接字
        #调用函数处理客户端请求
        client_handle(c)
        #子进程处理完客户端请求一定要退出
        c.close()
        sys.exit(0)
    else:
        c.close() #父进程不需要和客户端交互
        continue

## tftp文件服务器
- 功能要求:
  - 客户端有简单的命令提示界面
  - 根据提示选择相应的功能
    1. 查看服务器文件库的文件列表(只显示普通文件)
    2. 下载其中的某个文件到本地
    3. 可以将本地文件上传以文件库中
    4. 退出
    
  - 服务器需求:
    1. 处理客户端各种请求
    2. 允许多个客户端同时访问操作
   
- 技术分析
  - 用什么套接字
  - 需要什么服务器模型
  - 需要注意的问题
  - 数据如何传输
- 设计整体结构
  - 要求封装为类,将功能函数写类中
```python
class TFTPServer():
    def __init__():
    def do_list():
    def do_get():
    def do_put():
main()    #函数控制整体流程
main（）函数控制整体流程
'''

'''
```

In [1]:
# ftp_server
from socket import *
import os 
import signal 
import sys 
import time 

#文件库
FILE_PATH = "/home/tarena/"

#服务器功能
class TftpServer(object):
    def __init__(self,connfd):
        self.connfd = connfd 
    def handler(self):
        while True:
            data = self.connfd.recv(1024).decode()
            if not data or data[0] == 'Q':
                print(self.connfd.getpeername,"客户端退出")
                self.connfd.close()
                sys.exit(0)
            elif data[0] == "L":
                self.do_list()
            elif data[0] == "G":
                filename = data.split(' ')[-1]
                self.do_get(filename)
            elif data[0] == "P":
                filename = data.split(' ')[-1]
                self.do_put(filename)
                

    def do_list(self):
        #获取列表
        file_list = os.listdir(FILE_PATH)
        if not file_list:
            self.connfd.send("文件库为空".encode())
            return
        else:
            self.connfd.send(b'OK')
            time.sleep(0.1)
        files = ""
        for file in file_list:
            if file[0] != '.' and \
            os.path.isfile(FILE_PATH + file):
                files = files + file + '#'
        #发送给客户端
        self.connfd.send(files.encode())


    def do_get(self,filename):
        try:
            fd = open(FILE_PATH + filename,'rb')
        except:
            self.connfd.send("文件不存在".encode())
            return
        self.connfd.send(b'OK')
        time.sleep(0.1)

        #发送文件
        while True:
            data = fd.read(1024)
            if not data:
                time.sleep(0.1)
                self.connfd.send(b'##')
                break
            self.connfd.send(data)
            
        
        print("文件发送完成")


    def do_put(self,filename):
        try:
            fd = open(FILE_PATH + filename,'wb')
        except:
            self.connfd.send("无法上传".encode())
            return
        self.connfd.send(b'OK')

        while True:
            data = self.connfd.recv(1024)
            if data == b"##":
                break
            fd.write(data)
        fd.close()
        print("上传完毕")

#控制程序流程　创建套接字，接收链接，创建子进程
def main():
    HOST = "0.0.0.0"
    PORT = 8888
    ADDR = (HOST,PORT)

    s = socket()
    s.setsockopt(SOL_SOCKET,SO_REUSEADDR,1)
    s.bind(ADDR)
    s.listen(5)

    print("Listen to port 8888....")
    signal.signal(signal.SIGCHLD,signal.SIG_IGN)

    while True:
        try:
            connfd,addr = s.accept()
        except KeyboardInterrupt:
            s.close()
            sys.exit("退出服务器")
        except Exception as e:
            print(e)
            continue 
        print("客户端登录：",addr)

        #创建子进程
        pid = os.fork()
        if pid < 0:
            print("create process failed")
            connfd.close()
            continue
        elif pid == 0:
            s.close()
            tftp = TftpServer(connfd)
            tftp.handler()
        else:
            connfd.close()
            continue

if __name__ == "__main__":
    main()

In [ ]:
# ftp_client
from socket import *
import sys 
import time 

#功能封装为类
class TftpClient(object):
    def __init__(self,s):
        self.sockfd = s

    def do_list(self):
        self.sockfd.send(b"L")  #发送请求类型
        #等待服务器确认
        data = self.sockfd.recv(1024).decode()

        if data == 'OK':
            data = self.sockfd.recv(4096).decode()
            files = data.split('#')
            for file in files:
                print(file)
            print("%%%%%%文件展示完毕%%%%%%\n")
        else:
            #失败原因
            print(data)

    def do_get(self,filename):
        self.sockfd.send(('G ' + filename).encode())
        data = self.sockfd.recv(1024).decode()
        if data == 'OK':
            fd = open(filename,'wb')
            while True:
                data = self.sockfd.recv(1024)                
                if data == b"##":
                    break                   
                fd.write(data)
            fd.close()
            print("%s 下载完成\n"%filename)
        else:
            print(data)


    def do_put(self,filename):
        try:
            fd = open(filename,'rb')
        except:
            print("上传文件不存在")
            return

        self.sockfd.send(("P " + filename).encode())
        data = self.sockfd.recv(1024).decode()
        if data == 'OK':
            while True:
                data = fd.read(1024)
                if not data:
                    time.sleep(0.1)
                    self.sockfd.send(b'##')
                    break
                self.sockfd.send(data)
            print("%s 文件上传完毕"%filename)
        else:
            print(data)


    def do_quit(self):
        self.sockfd.send(b'Q') 

#套接字连接
def main():
    if len(sys.argv) < 3:
        print("argv is error")
        return 
    HOST = sys.argv[1]
    PORT = int(sys.argv[2])
    ADDR = (HOST,PORT)

    s = socket()
    s.connect(ADDR)
    tftp = TftpClient(s)

    while True:
        print("")
        print("========命令选项=======")
        print("******** list *******")
        print("****** get file *****")
        print("****** put file *****")
        print("******** quit *******")
        print("======================")

        cmd = input("请输入命令>>")

        if cmd.strip() == 'list':
            tftp.do_list()
        elif cmd[:3] == "get":
            filename = cmd.split(' ')[-1]
            tftp.do_get(filename)
        elif cmd[:3] == "put":
            filename = cmd.split(' ')[-1]
            tftp.do_put(filename) 
        elif cmd.strip() == "quit":
            tftp.do_quit()
            sys.exit("欢迎使用")
        else:
            print("请输入正确的命令！！！")
            continue

if __name__ == "__main__":
    main()